## Adding a timestamp when you index documents

This notebook shows how you can add custom fields and actions to run automatically when you ingest data with the Python client. 

Using the concepts of: [the ingest pipeline](https://www.elastic.co/guide/en/elasticsearch/reference/8.12/ingest.html) and [the set processor.](https://www.elastic.co/guide/en/elasticsearch/reference/current/set-processor.html)

In [2]:
from getpass import getpass  
from elasticsearch import Elasticsearch

# Prompt the user to enter their Elastic Cloud ID and API Key securely
ELASTIC_CLOUD_ID = getpass("Elastic Cloud ID: ")
ELASTIC_API_KEY = getpass("Elastic API Key: ")

# Create an Elasticsearch client using the provided credentials
es = Elasticsearch(
    cloud_id=ELASTIC_CLOUD_ID,  # cloud id can be found under deployment management
    api_key=ELASTIC_API_KEY, # your username and password for connecting to elastic, found under Deplouments - Security
)

In [3]:
index_name = "test_timestamp"

mappings = {
    "properties" : {
        "foo" : {
            "type" : "keyword",
            "type" : "text"
        },
        "created_at": {
            "type": "date" 
        },
    }
}

settings = {
    "index.default_pipeline" : "ingest_with_dates"
}

es.ingest.put_pipeline(
    id="ingest_with_dates", 
    processors=[
    {
        "set": {
            "field": "created_at",
            "value": "{{_ingest.timestamp}}"
        }
    }]
)

es.indices.create(index=index_name, mappings=mappings, settings=settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'test_timestamp'})

In [4]:
es.index(
    index=index_name,
    id=0,
    document={
        "foo": "bar",
    },
)

ObjectApiResponse({'_index': 'test_timestamp', '_id': '0', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 0, '_primary_term': 1})

In [5]:
query={
    "match": {
        "foo": "bar"
    }
}

response = es.search(index=index_name, query=query)

In [7]:
for hit in response["hits"]["hits"]:
    print(hit['_source'])

{'created_at': '2024-03-12T12:50:02.626995027Z', 'foo': 'bar'}
